In [1]:
!pip install -r requirements.txt -q
!pip install cloudflared -q
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

!cloudflared --version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 136.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 135.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.1 MB/s eta 0:00:00
Selecting previously unselected package cloudflared.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.9.1) ...
Setting up cloudflared (2025.9.1) ...
Processing triggers for man-db (2.10.2-1) ...
cloudflared version 2025.9.1 (built 2025-09-22-13:28 UTC)


In [2]:
import sqlite3
conn = sqlite3.connect("/content/db/finance.db")  # or your DB path
cursor = conn.cursor()

sample_expenses = [
    ("2025-10-02", "Food", 500),
    ("2025-10-05", "Transport", 200),
    ("2025-10-10", "Shopping", 1200),
]


cursor.executemany("INSERT INTO expenses (date, category, amount) VALUES (?, ?, ?)", sample_expenses)
conn.commit()
conn.close()


In [5]:
import threading, os

# Kill anything on port 8000 (just in case)
os.system("kill -9 $(lsof -t -i:8000) 2>/dev/null")

# Run uvicorn in background
def run_app():
    os.system("uvicorn server:app --host 0.0.0.0 --port 8000 --reload")

thread = threading.Thread(target=run_app)
thread.start()

# Start cloudflared tunnel
!cloudflared tunnel --url http://localhost:8000 --no-autoupdate

2025-10-04T11:37:59Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-04T11:37:59Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-04T11:38:04Z INF +--------------------------------------------------------------------------------------------+
2025-10-04T11:38:04Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-04T11:38:04Z INF |  https://reflection-tobacco-involved-monster.trycloudf